In [1]:
# Next code is used to personalize the slides format. 
# The customization of the slides was not working for RISE after resuming the presentation, check issue 
# https://github.com/damianavila/RISE/issues/225
#from traitlets.config.manager import BaseJSONConfigManager
#path = "/Users/camilocardona/anaconda/etc/jupyter/nbconfig"
#cm = BaseJSONConfigManager(config_dir=path)
#cm.update('livereveal', {
#              'theme': 'serif',
#              'transition': 'None',
#              'start_slideshow_at': 'selected',
#})

In [3]:
import ipywidgets as widgets
import json
import qgrid
from IPython.display import display 
from IPython.display import Image
import pandas as pd
import matplotlib.pyplot as plt
import ipympl
%matplotlib widget
from json_browswer import json_browser

In [3]:
# Please decompress before running this command!
traffic_matrix = pd.read_pickle("traffic_matrix.pickle")

<h1>Construyendo Dashboards interactivos en
Jupyter Notebooks</h1>
<p><strong>Juan Camilo Cardona</strong></p>
<strong>NTT</strong>
<p>&nbsp;</p>
<p>&nbsp;</p>


# Objetivos de la presentación

* Demostrar como construir dashboard interactivos en jupyter.
    * Con Botones, cajas de texto, grafiquitos que cambian, etc. 
* Ilustrar la flexibilidad del sistema para crear dashboards relativamente complejos 
* Discutir los alcances potenciales de la solución
    * Aunque no sea el framework de UI ideal, puede ser muy util en ciertos casos. Prototipos o proyectos con exigencias sencillas

# Cúal es el problema?

* Si trabajamos con datos, eventualmente haremos parte de un proyecto que requiera un ambiente grafico interactivo  (i.e. UI) con el usuario final:
    * Modificar parámetros de optimización
    * Explorar los datos, agrupar, filtrar, etc.
    * Operar directamente el sistema en un ambiente grafico

# Cúal es el problema?

* Los frameworks de UI son muy potentes, pero pocas veces tenemos control sobre ellos:
    * E.g. Nadie en el equipo conoce de UI
    * E.g. Necesitamos dinero para contratar a los expertos de UI, pero necesitamos un prototipo para venderlo 
    * E.g. Tenemos gente de UI, pero no es facil trabajar con ellos para hacer prototipos y recibir feedback rápido

# Cómo resolver el problema?

* Aprender un framework de UI
    * Pros? Puede ser bueno para mi carrera, me gustaria aprender del tema, podre ser útil en el equipo.
    * Cons? No tengo tiempo, no me agrada, no se cual tecnología escoger.


# Cómo resolver el problema?
* Utilizar a jupyter directamente como dashboard
    * Pros? Me permite fácilmente tener algo listo para evaluación del grupo o uso en proyectos no muy exigentes.
    * Cons? (Opinión  propia) No brinda todas las garantías para ser el UI del diseño final.

# Propuesta

* Jupyter
* Ipywidgets (jupyter-widgets). Otros widgets especiales (e.g. Jupyter-matplotlib).
* Voila

<span style="color:#800000">(Bokeh dashboards, Dash u otras tecnologias pueden ofrecer una alternativa)</span>


# Ipywidgets (jupyter-widgets)
* Provee elementos (widgets) interactivos para Jupyter
    * Including Buttons, Text fields, HTML fields, etc.

In [1]:
button_widget = widgets.Button(description="Un Boton")
display(button_widget)

NameError: name 'widgets' is not defined

In [5]:
widgets.Text(value="Text box 2")

Text(value='Text box 2')

In [6]:
slider = widgets.IntSlider()
display(slider)

IntSlider(value=0)

# Ipywidgets (jupyter-widgets)
* Existen muchos otros. Miraremos algunos en el resto de la presentación.
* Listas de widgets:
    * https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
    * https://jupyter.org/widgets
* Existen proyectos independientes para otros widgets   

# Container widgets

* Existen widgets especiales para "contener" los widgets (container widgets).
    * E.g. AppLayout, Grids. Boxes, Tabs, o acordeon.
* Los contenedores también  son widgets. Por lo tanto pueden tener otros como hijos
    * Asi que se pueden componer "dashboards" complejos con jerarquías  de contenedores.


In [7]:
# redefine the buttons and text box, to avoid any problem.
button_1 = widgets.Button(description="Button1")
text_1 = widgets.Text(value="text1")
button_2 = widgets.Button(description="Button2")
text_2 = widgets.Text(value="text2")

In [8]:
# button_1, button_2, text_1, text_2 were defined in other cell.
box_1 = widgets.VBox((button_1, text_1))
box_2 = widgets.HBox((button_2, text_2))
tab_widget = widgets.Tab((box_1, box_2), _titles={0: "V", 1:"H"})
display(tab_widget)

In [9]:
# Vamos ahora a incrustrar el anterior componente, y otros más, en una appLayout.
header = widgets.Button(description="header")
footer = widgets.Button(description="footer")
left = widgets.Button(description="left")
right = widgets.Button(description="right")
app = widgets.AppLayout(center=tab_widget, header=header, 
                        footer=footer, left_sidebar=left, right_sidebar=right,
                        align_items='center'
                       )
display(app)


AppLayout(children=(Button(description='header', layout=Layout(grid_area='header'), style=ButtonStyle()), Butt…

In [10]:
widgets.Tab((widgets.Button(description="Titulo"), app), _titles={0: "Titulo", 1:"APP"})

# Cómo ajustar visualmente los widgets?

* Los widgets (probablemente casi todos) aceptan un layout que permite controlar su tamaño, color, etc.
* El layout también puede controlar como los widgets contenedores agrupan a sus hijos.
* Tip: Puede ser frustrante obtener lo que deseas.

In [11]:
widgets.Button(description="Long button", layout=widgets.Layout(width='400px'))

Button(description='Long button', layout=Layout(width='400px'), style=ButtonStyle())

In [12]:
lo = widgets.Layout(width='600px', justify_content='space-around', border='solid')
widgets.HBox((widgets.Button(description="B1"), widgets.Button(description="B2")), layout=lo)

# Interación con los widgets
* Se definen callbacks. Es decir, se registran funciones que se invocan cuando se ejecuta un acción.


In [13]:
boton = widgets.Button(description="Habilitar texto")
caja_de_texto = widgets.Text(value="Caja de texto")

def habilitar_texto(caller=None):
    caja_de_texto.disabled = False
        
def desabilitar_texto(caller=None):
    caller['owner'].disabled = True

boton.on_click(habilitar_texto)
caja_de_texto.observe(desabilitar_texto, names="value")

display(widgets.HBox((boton, caja_de_texto)))

# Como representar figuras?

* Una opción es [**Jupyter-matplotlib**](https://github.com/matplotlib/jupyter-matplotlib) 
    * Un widget para figuras matplotlib
* Existen alternativas como [bqplot](https://github.com/bloomberg/bqplot).
* Requiere instalación adicional, y puede requerir de un poco más de conocimiento

In [4]:
figure, axis = plt.subplots()
figure.set_size_inches([ 6,  3 ])
button_mpl = widgets.Button(description="Clear Figure")
button_mpl.on_click(lambda _: (axis.clear(), plt.draw_all()))
button_mpl_2 = widgets.Button(description="Draw Figure")
button_mpl_2.on_click(lambda _: (axis.plot([0, 1, 2, 1]), plt.draw_all()))
axis.plot([0, 1, 2, 1])
box = widgets.HBox(children=(widgets.HBox((figure.canvas,)),button_mpl, button_mpl_2,))
display(box)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# Pongamos lo anterior en otro widget para demostrar de nuevo la componibilidad
app = widgets.TwoByTwoLayout(top_left=button_mpl,
                     bottom_left=button_mpl_2,
                     bottom_right=figure.canvas,
                     align_items="center",
                     height='400px')
display(app)

TwoByTwoLayout(children=(Button(description='Clear Figure', layout=Layout(grid_area='top-left'), style=ButtonS…

# Como representar dataframes
* Utilizando el widget HTML.
    * Junto con [df styling](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html) o con df.to_html(). 
* Utilizando el widget Output.
* Para mayor funcionalidad, hay otras alterativas:
    * https://github.com/quantopian/qgrid
    * https://dgothrek.gitlab.io/ipyaggrid/
    * https://github.com/QuantStack/ipysheet

In [17]:
df = traffic_matrix[:10]
df2 = traffic_matrix[10:20]
from  traffic_browser import hover
wg1 = widgets.HTML(df.style.set_table_attributes('class=\"table\"').set_table_styles([hover()]).render())
wg2 = widgets.HTML(df2.style.set_table_attributes('class=\"table\"').set_table_styles([hover()]).render())
display(wg2)

HTML(value='<style  type="text/css" >\n    #T_37a662e2_49c0_11ea_b066_3af9d378ec91 tr:hover {\n          backg…

In [18]:
box_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='100%')
box_layout = widgets.Layout(display='flex',
                    border='solid',
                    width='100%')
box = widgets.HBox([wg1, wg2], layout=box_layout)
display(box)

In [19]:
# Tomado de https://stackoverflow.com/questions/26873127/show-dataframe-as-table-in-ipython-notebook
# create output widgets
widget1 = widgets.Output()
widget2 = widgets.Output()
# render in output widgets
with widget1:
    display(df)
with widget2:
    display(df2)
# create HBox
hbox = widgets.HBox([widget1, widget2])
# render hbox
hbox

In [20]:
# Ejemplo de uso de qgrid (se debe instalar antes).
# Termino escondida porque afectaba otros widgets.
#gr = qgrid.show_grid(df)
#display(widgets.HBox(children=[gr]))

# Ejemplos: Navegador de JSON

In [21]:
json_ejemplo = '''{
  "Nivel1": {
      "Nivel2": {
          "Valor1": "Nivel2-Valor1",
          "Valor2": "Nivel2-Valor2",
          "Nivel3": {
              "Valor1": "Nivel3-Valor1"
          }
      },
      "Valor1": "Nivel1-Valor1"
  }
}'''

In [22]:
json_browser(json.loads(json_ejemplo))

In [23]:
# The next jsons are "fake" configurations for all routers.
# I took the model from https://github.com/CiscoDevNet/openconfig-getting-started/blob/master/models/bgp/
bgp_json = '''
{
 "bgp:bgp": {
  "global": {
   "state": {
    "as": 65001,
    "total-paths": 2,
    "total-prefixes": 2
   },
   "afi-safis": {
    "afi-safi": [
     {
      "afi-safi-name": "ipv4-unicast",
      "state": {
       "afi-safi-name": "ipv4-unicast",
       "enabled": true,
       "total-paths": 2,
       "total-prefixes": 2
      }
     }
    ]
   }
  },
  "peer-groups": {
   "peer-group": [
    {
     "peer-group-name": "IBGP",
     "state": {
      "peer-group-name": "IBGP",
      "peer-as": 65001
     },
     "transport": {
      "state": {
       "local-address": "Loopback0"
      }
     },
     "afi-safis": {
      "afi-safi": [
       {
        "afi-safi-name": "ipv4-unicast",
        "state": {
         "afi-safi-name": "ipv4-unicast",
         "enabled": true
        },
        "apply-policy": {
         "state": {
          "export-policy": [
           "POLICY2"
          ]
         }
        }
       }
      ]
     }
    }
   ]
  },
  "neighbors": {
   "neighbor": [
    {
     "neighbor-address": "172.16.255.3",
     "state": {
      "neighbor-address": "172.16.255.3",
      "peer-group": "IBGP",
      "queues": {
       "input": 0,
       "output": 0
      },
      "session-state": "bgp-st-estab",
      "supported-capabilities": [
       "MPBGP"
      ],
      "messages": {
       "sent": {
        "NOTIFICATION": 0,
        "UPDATE": 1
       },
       "received": {
        "NOTIFICATION": 0,
        "UPDATE": 3
       }
      }
     },
     "transport": {
      "state": {
       "local-port": 21344,
       "remote-address": "172.16.255.3",
       "remote-port": 179
      }
     },
     "timers": {
      "state": {
       "negotiated-hold-time": 180
      }
     },
     "afi-safis": {
      "afi-safi": [
       {
        "afi-safi-name": "ipv4-unicast",
        "state": {
         "active": true,
         "prefixes": {
          "received": 2,
          "sent": 0
         }
        }
       }
      ]
     },
     "graceful-restart": {
      "state": {
       "peer-restart-time": 120
      }
     }
    }
   ]
  }
 }
}
'''

In [24]:
json_browser(json.loads(bgp_json))

# Explorador de tráfico

In [25]:
import traffic_browser
traffic_browser.ts_widget(traffic_matrix, top_flows_to_show=5, align_vertically=False, 
          time_column="TIME", value_column="BW")

Hay muchos más ejemplos en https://voila-gallery.org/

(Si hay tiempo y forma miraremos el dashboard de https://github.com/seidlr/voila-interactive-football-pitch)

# Cómo publicar los dashboards?

* [Voilà](https://github.com/voila-dashboards/voila) es la proyecto recomendado para publicar dashboards.
* Para proyectos muy pequeños, o personales, se pueden ejecutar directamente el notebook o el jupyterlab.


# Voilà

* Voilà ejecuta el notebook, y representa el **output** de las celdas del notebook. Ignora las que no tienen ningún output.
    * Por ahora, Voilà representa las celdas solo en el orden original.
    * Hay planes para aumentar la flexibilidad de posición de celdas, pero no esta listo aún. 
        * Mirar [voila-gridstack](https://github.com/voila-dashboards/voila-gridstack)
* Cada usuario obtiene un kernel dedicado.

# Experiencia personal

* Muy adecuado para prototipos y proyectos de alcance corto.
* Suficientes opciones para mostrar texto, graficos, y tablas
* Los he utilizado para:
    * Explorar datos jerarquicos
    * Crear pequeños prototipos de productos

# Conclusiones
En prototipos, lo perfecto es el enemigo del progreso.

Algunos de estas librerias/widgets pueden tener limitantes técnicas.

Utilicen suficiente tiempo para expresar ideas, pero no exageren.


# Preguntas?
Código en https://github.com/jccardonar/pycon_colombia2020
    
Referencias futuras:
    * https://twitter.com/SylvainCorlay
    * https://gitter.im/QuantStack/Lobby